In [2]:
# imports
import os
import pandas as pd

from datetime import datetime, timedelta
from s3.s3 import bucket
from dotenv import load_dotenv
from database.db import database

load_dotenv(".env")

True

In [3]:
# load csv
orders_df = pd.read_csv("downloads/orders.csv")
reviews_df = pd.read_csv("downloads/reviews.csv")
shipments_df = pd.read_csv("downloads/shipment_deliveries.csv")

# View CSV Files

In [15]:
orders_df.head(20)

,order_id,customer_id,order_date,product_id,unit_price,quantity,total_price
0,1,5,2022-07-13,24,139,10,1390
1,2,14,2021-04-06,2,273,4,1092
2,3,17,2022-07-29,20,253,9,2277
3,4,14,2022-08-27,8,334,1,334
4,5,25,2021-12-15,6,334,3,1002
5,6,7,2021-04-15,25,144,7,1008
6,7,4,2021-09-28,12,876,1,876
7,8,24,2021-05-08,15,994,7,6958
8,9,3,2022-01-06,18,641,4,2564
9,10,23,2021-08-22,3,841,5,4205


In [16]:
reviews_df.head(20)

,review,product_id
0,1,21
1,3,1
2,2,8
3,1,5
4,5,22
5,5,3
6,1,14
7,1,1
8,2,7
9,5,17


In [17]:
shipments_df.head(20)

,shipment_id,order_id,shipment_date,delivery_date
0,1,1,2022-07-14,NaN
1,2,2,NaN,NaN
2,3,3,2022-07-31,2022-08-03
3,4,4,2022-09-02,2022-09-05
4,5,5,2021-12-19,2021-12-20
5,6,6,2021-04-17,2021-04-18
6,7,7,NaN,NaN
7,8,8,2021-05-10,2021-05-13
8,9,9,NaN,NaN
9,10,10,NaN,NaN


In [110]:
# Connect to database
db = database()
db_conn = db.conn()
db_cursor = db.create_cursor(db_conn)
staging = os.getenv("STAGING")
analytics = os.getenv("ANALYTICS")

# # Make Query
# data = db.fetch_data(db_cursor,"*",f"{staging}.orders",secondary_tablename="if_common.dim_dates",all=True,filtered=True,filter="if_common.dim_dates.working_day = False AND 1 <= if_common.dim_dates.day_of_the_week_num AND 5 >= if_common.dim_dates.day_of_the_week_num",join=True,join_condition=f"if_common.dim_dates.calendar_dt = {staging}.orders.order_date")
# data

In [29]:
data_columns = ["db_id"] + list(orders_df.columns) + ["calendar_dt","year_num","month_of_the_year_num","day_of_the_month_num","day_of_the_week_num","working_day"]
data_columns

['db_id',
 'order_id',
 'customer_id',
 'order_date',
 'product_id',
 'unit_price',
 'quantity',
 'total_price',
 'calendar_dt',
 'year_num',
 'month_of_the_year_num',
 'day_of_the_month_num',
 'day_of_the_week_num',
 'working_day']

In [79]:
data_df = pd.DataFrame(data,columns=data_columns)
data_df

,db_id,order_id,customer_id,order_date,product_id,unit_price,quantity,total_price,calendar_dt,year_num,month_of_the_year_num,day_of_the_month_num,day_of_the_week_num,working_day
0,24,24,22,2022-07-04,24,818,5,4090,2022-07-04,2022,7,4,1,False
1,68,68,21,2022-03-29,9,614,7,4298,2022-03-29,2022,3,29,2,False
2,75,75,14,2021-10-01,22,213,2,426,2021-10-01,2021,10,1,5,False
3,79,79,12,2022-02-14,21,745,9,6705,2022-02-14,2022,2,14,1,False
4,89,89,10,2021-01-01,13,361,5,1805,2021-01-01,2021,1,1,5,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,9812,9809,8,2021-03-29,12,391,3,1173,2021-03-29,2021,3,29,1,False
170,9821,9818,22,2021-04-01,19,447,7,3129,2021-04-01,2021,4,1,4,False
171,9826,9823,5,2021-11-11,8,549,8,4392,2021-11-11,2021,11,11,4,False
172,9886,9883,1,2021-01-01,23,50,6,300,2021-01-01,2021,1,1,5,False


In [80]:
todays_date = datetime.utcnow().date()
last_year_limit = todays_date - timedelta(days=365)
last_year_limit

datetime.date(2021, 9, 26)

In [81]:
for i in range(len(data)):
    if (data_df["order_date"][i] < last_year_limit):
        data_df.drop(i,inplace=True)

In [82]:
data_df = data_df.reset_index(drop=True)

In [86]:
months = {1:0,2:0,3:0,4:0,5:0,6:0,7:0,8:0,9:0,10:0,11:0,12:0}

for i in range(len(data_df)):
    monthKey = int(data_df["month_of_the_year_num"][i])
    months[monthKey] = months[data_df["month_of_the_year_num"][i]] + 1

In [91]:
 mnths = {
            "tt_order_hol_jan":[months[1]],
            "tt_order_hol_feb":[months[2]],
            "tt_order_hol_mar":[months[3]],
            "tt_order_hol_apr":[months[4]],
            "tt_order_hol_may":[months[5]],
            "tt_order_hol_jun":[months[6]],
            "tt_order_hol_jul":[months[7]],
            "tt_order_hol_aug":[months[8]],
            "tt_order_hol_sep":[months[9]],
            "tt_order_hol_oct":[months[10]],
            "tt_order_hol_nov":[months[11]],
            "tt_order_hol_dec":[months[12]]
            }
mnths

{'tt_order_hol_jan': [0],
 'tt_order_hol_feb': [17],
 'tt_order_hol_mar': [13],
 'tt_order_hol_apr': [12],
 'tt_order_hol_may': [0],
 'tt_order_hol_jun': [0],
 'tt_order_hol_jul': [13],
 'tt_order_hol_aug': [17],
 'tt_order_hol_sep': [0],
 'tt_order_hol_oct': [16],
 'tt_order_hol_nov': [12],
 'tt_order_hol_dec': [0]}

In [92]:
df_dict = pd.DataFrame.from_dict(mnths)

In [93]:
df_dict

,tt_order_hol_jan,tt_order_hol_feb,tt_order_hol_mar,tt_order_hol_apr,tt_order_hol_may,tt_order_hol_jun,tt_order_hol_jul,tt_order_hol_aug,tt_order_hol_sep,tt_order_hol_oct,tt_order_hol_nov,tt_order_hol_dec
0,0,17,13,12,0,0,13,17,0,16,12,0


In [119]:
# Connect to database
db = database()
db_conn = db.conn()
db_cursor = db.create_cursor(db_conn)
staging = os.getenv("STAGING")
analytics = os.getenv("ANALYTICS")

# Fetch Data
task2_data = db.fetch_data(db_cursor,"*",f"{staging}.orders",secondary_tablename=f"{staging}.shipment_deliveries",all=True,join=True,join_condition=f"{staging}.shipment_deliveries.order_id = {staging}.orders.order_id",filtered=True,filter=f"{staging}.shipment_deliveries.delivery_date = '-' AND {staging}.shipment_deliveries.shipment_date != '-'")

In [120]:
data_columns = ["oid"] + list(orders_df.columns) + ["sid","shipment_id","order_id","shipment_date","delivery_date"]
data_columns

['oid',
 'order_id',
 'customer_id',
 'order_date',
 'product_id',
 'unit_price',
 'quantity',
 'total_price',
 'sid',
 'shipment_id',
 'order_id',
 'shipment_date',
 'delivery_date']

In [121]:
task2_data_df = pd.DataFrame(task2_data,columns=data_columns)
task2_data_df

,oid,order_id,customer_id,order_date,product_id,unit_price,quantity,total_price,sid,shipment_id,order_id,shipment_date,delivery_date
0,1,1,5,2022-07-13,24,139,10,1390,1,1,1,2022-07-14,-
1,21,21,3,2021-04-23,25,206,3,618,21,21,21,2021-04-27,-
2,24,24,22,2022-07-04,24,818,5,4090,24,24,24,2022-07-10,-
3,52,52,16,2021-06-11,17,108,9,972,52,52,52,2021-06-17,-
4,65,65,20,2021-07-28,7,753,5,3765,65,65,65,2021-08-02,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1041,9978,9975,25,2022-04-22,13,326,4,1304,9975,9975,9975,2022-04-27,-
1042,9982,9979,5,2021-02-15,17,439,1,439,9979,9979,9979,2021-02-17,-
1043,9983,9980,25,2022-02-02,21,287,5,1435,9980,9980,9980,2022-02-03,-
1044,9988,9985,11,2022-01-04,5,788,2,1576,9985,9985,9985,2022-01-05,-


In [125]:
late_shipment_counter = 0
for i in range(len(task2_data_df)):
    order_date = task2_data_df["order_date"][i]
    shipment_date_str = task2_data_df["shipment_date"][i]
    shipment_date = datetime.strptime(shipment_date_str,"%Y-%m-%d").date()
    diff = shipment_date - order_date
    if diff >= timedelta(days=6):
        late_shipment_counter += 1
late_shipment_counter

175

In [126]:
# Connect to database
db = database()
db_conn = db.conn()
db_cursor = db.create_cursor(db_conn)
staging = os.getenv("STAGING")
analytics = os.getenv("ANALYTICS")

# Fetch Data
task3_data = db.fetch_data(db_cursor,"*",f"{staging}.orders",secondary_tablename=f"{staging}.shipment_deliveries",all=True,join=True,join_condition=f"{staging}.shipment_deliveries.order_id = {staging}.orders.order_id",filtered=True,filter=f"{staging}.shipment_deliveries.delivery_date = '-' AND {staging}.shipment_deliveries.shipment_date = '-'")

In [127]:
data_columns = ["oid"] + list(orders_df.columns) + ["sid","shipment_id","order_id","shipment_date","delivery_date"]
data_columns

['oid',
 'order_id',
 'customer_id',
 'order_date',
 'product_id',
 'unit_price',
 'quantity',
 'total_price',
 'sid',
 'shipment_id',
 'order_id',
 'shipment_date',
 'delivery_date']

In [128]:
task3_data_df = pd.DataFrame(task3_data,columns=data_columns)
task3_data_df

,oid,order_id,customer_id,order_date,product_id,unit_price,quantity,total_price,sid,shipment_id,order_id,shipment_date,delivery_date
0,2,2,14,2021-04-06,2,273,4,1092,2,2,2,-,-
1,7,7,4,2021-09-28,12,876,1,876,7,7,7,-,-
2,9,9,3,2022-01-06,18,641,4,2564,9,9,9,-,-
3,10,10,23,2021-08-22,3,841,5,4205,10,10,10,-,-
4,11,11,3,2022-01-25,8,334,8,2672,11,11,11,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6752,9997,9994,20,2022-01-08,5,915,3,2745,9994,9994,9994,-,-
6753,9998,9995,7,2022-06-03,9,553,1,553,9995,9995,9995,-,-
6754,9999,9996,25,2022-07-30,12,863,4,3452,9996,9996,9996,-,-
6755,10001,9998,17,2022-07-23,12,114,9,1026,9998,9998,9998,-,-


In [129]:
undelivered_shipment_counter = 0
for i in range(len(task2_data_df)):
    order_date = task2_data_df["order_date"][i]
    todays_date = datetime.utcnow().date()
    diff = todays_date - order_date
    if diff >= timedelta(days=15):
        undelivered_shipment_counter += 1
undelivered_shipment_counter

1046

In [31]:
# Connect to database
db = database()
db_conn = db.conn()
db_cursor = db.create_cursor(db_conn)
staging = os.getenv("STAGING")
analytics = os.getenv("ANALYTICS")

# Fetch review data
review_data = db.fetch_data(db_cursor,"*",f"{staging}.reviews")

In [32]:
review_count_dict = {}
review_points_sum = {}
review_points_distribution = {}

for review in review_data:
    if f"id_{review[2]}" in review_count_dict.keys():
        review_count_dict[f"id_{review[2]}"] = review_count_dict[f"id_{review[2]}"] + 1
    else:
        review_count_dict[f"id_{review[2]}"] = 1

    if f"id_{review[2]}" in review_points_sum.keys():
        review_points_sum[f"id_{review[2]}"] = review_points_sum[f"id_{review[2]}"] + review[1]
    else:
        review_points_sum[f"id_{review[2]}"] = review[1]

    if f"id_{review[2]}_{review[1]}" in review_points_distribution.keys():
        review_points_distribution[f"id_{review[2]}_{review[1]}"] = review_points_distribution[f"id_{review[2]}_{review[1]}"] + 1
    else:
        review_points_distribution[f"id_{review[2]}_{review[1]}"] = 1

# Highest Reviewed Product
import math
index = list(review_count_dict.values()).index(max(list(review_count_dict.values())))
highest_reviewed = list(review_count_dict.keys())[index]
highest_reviewed_id = highest_reviewed.split("_")[-1]
review_sum = review_points_sum[highest_reviewed]
pct_one_star = (review_points_distribution[f"{highest_reviewed}_1"]/review_count_dict[highest_reviewed])*100
pct_two_star = (review_points_distribution[f"{highest_reviewed}_2"]/review_count_dict[highest_reviewed])*100
pct_three_star = (review_points_distribution[f"{highest_reviewed}_3"]/review_count_dict[highest_reviewed])*100
pct_four_star = (review_points_distribution[f"{highest_reviewed}_4"]/review_count_dict[highest_reviewed])*100
pct_five_star = (review_points_distribution[f"{highest_reviewed}_5"]/review_count_dict[highest_reviewed])*100
product_data = db.fetch_data(db_cursor,"product_name","if_common.dim_products",filtered=True,filter=f"product_id = {highest_reviewed_id}",all=False,one=True)
product_name = product_data[0]


In [33]:
# Fetch order data for best performing product
order_data = db.fetch_data(db_cursor,"*",f"{staging}.orders",filtered=True,filter=f"product_id = {highest_reviewed_id}")

# Find date of highest data
highest_order_date = datetime.strptime("0001-01-01","%Y-%m-%d").date()
highest_order = 0
for order in order_data:
    if (order[6] >= highest_order) and (order[3] >= highest_order_date):
        highest_order_date = order[3]
        highest_order = order[6]

In [34]:
# Check date is public holiday or not with dim_dates
calender_data = db.fetch_data(db_cursor,"day_of_the_week_num,working_day","if_common.dim_dates",filtered=True,filter=f"calendar_dt = '{highest_order_date}'")
day_of_the_week,working_day = calender_data[0]

if (day_of_the_week in range(1,6)):
    if (working_day == True):
        is_public_holiday = False
    else:
        is_public_holiday = True

In [35]:
# Early and late shipments
shipments_data = db.fetch_data(
    db_cursor,
    "*",
    f"{staging}.orders",
    secondary_tablename=f"{staging}.shipment_deliveries",
    all=True,
    join=True,
    join_condition=f"{staging}.shipment_deliveries.order_id = {staging}.orders.order_id",
    filtered=True,
    filter=f"{staging}.shipment_deliveries.shipment_date != '-' AND {staging}.orders.product_id = {highest_reviewed_id}")

data_columns = ["oid"] + list(orders_df.columns) + ["sid","shipment_id","order_id","shipment_date","delivery_date"]

shipments_data_df = pd.DataFrame(shipments_data,columns=data_columns)

In [36]:
total_shipments = len(shipments_data)
early_shipment_count = 0
late_shipment_count = 0

for i in range(total_shipments):
    order_date = shipments_data_df["order_date"][i]
    shipment_date_str = shipments_data_df["shipment_date"][i]
    shipment_date = datetime.strptime(shipment_date_str,"%Y-%m-%d").date()
    diff = shipment_date - order_date
    if diff >= timedelta(days=6):
        late_shipment_count += 1
    else:
        early_shipment_count += 1

pct_late_shipment = (late_shipment_count/total_shipments)*100
pct_early_shipment = (early_shipment_count/total_shipments)*100

In [37]:
print(highest_reviewed_id)
print(product_name)
print(review_sum)
print(pct_one_star)
print(pct_two_star)
print(pct_three_star)
print(pct_four_star)
print(pct_five_star)
print(highest_order_date)
print(is_public_holiday)
print(pct_early_shipment)
print(pct_late_shipment)

23
Screwdriver
940
18.670886075949365
23.417721518987342
18.670886075949365
20.253164556962027
18.9873417721519
2022-08-18
False
81.88976377952756
18.11023622047244


In [13]:
order_data

[(96, 96, 7, datetime.date(2022, 5, 22), 23, 459, 4, 1836),
 (125, 125, 13, datetime.date(2021, 3, 19), 23, 656, 4, 2624),
 (126, 126, 4, datetime.date(2021, 5, 4), 23, 865, 10, 8650),
 (144, 144, 25, datetime.date(2021, 7, 16), 23, 271, 8, 2168),
 (160, 160, 10, datetime.date(2022, 4, 10), 23, 122, 4, 488),
 (179, 179, 8, datetime.date(2021, 11, 15), 23, 614, 1, 614),
 (182, 182, 17, datetime.date(2022, 3, 2), 23, 904, 8, 7232),
 (197, 197, 21, datetime.date(2022, 7, 9), 23, 86, 10, 860),
 (273, 273, 18, datetime.date(2022, 5, 28), 23, 622, 6, 3732),
 (302, 302, 22, datetime.date(2021, 4, 6), 23, 711, 7, 4977),
 (341, 341, 16, datetime.date(2022, 1, 12), 23, 996, 9, 8964),
 (359, 359, 23, datetime.date(2022, 1, 6), 23, 570, 4, 2280),
 (400, 400, 24, datetime.date(2022, 9, 4), 23, 535, 5, 2675),
 (420, 420, 11, datetime.date(2021, 4, 13), 23, 747, 3, 2241),
 (442, 442, 17, datetime.date(2021, 3, 21), 23, 989, 10, 9890),
 (450, 450, 12, datetime.date(2022, 7, 9), 23, 111, 2, 222),
 (461

In [97]:
db.close(db_conn)